In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import time

#model Definition
# Scale the input variables
scaler = MinMaxScaler()
# Build the ANN model
model = Sequential()
model.add(Dense(20, input_dim=1, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(3))  # Output layer with 3 neurons for rms 1, 2, and 3

# Load the data
data1 = {
    'Forces': ['10 N', '15 N', '20 N', '25 N', '35 N', '30 N', '40 N', '45 N', '50 N', '55 N', '60 N', '65 N', '70 N', '75 N'],
    'rms 1': [0.09, 0.22, 0.1, 0.19, 0.09, 0.11, 0.1, 0.08, 0.08, 0.09, 0.08, 0.07, 0.06, 0.05],
    'rms 2': [0.25, 0.29, 0.34, 0.2, 0.2, 0.14, 0.22, 0.12, 0.14, 0.09, 0.09, 0.06, 0.12, 0.1],
    'rms 3': [0.2, 0.14, 0.07, 0.11, 0.22, 0.13, 0.12, 0.11, 0.13, 0.09, 0.07, 0.1, 0.09, 0.05] }

df1 = pd.DataFrame(data1)

# Split the data into input (X) and output (y) variables
X1 = df1[['Forces']]
y1 = df1[['rms 1', 'rms 2', 'rms 3']]

# Convert the 'Forces' column to numeric values
X1['Forces'] = X1['Forces'].str.replace(' N', '').astype(int)



X_scaled1 = scaler.fit_transform(X1)



# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_scaled1, y1, epochs=200, batch_size=10, verbose=1)

# Function to predict rms based on force input
def predict_rms(force):
    force_scaled = scaler.transform([[force]])
    prediction = model.predict(force_scaled)[0]
    #return prediction
    mean_rms = np.mean(prediction, axis=0)
    return mean_rms

# Load the data
data2 = {
    'Forces': ['10 N', '15 N', '20 N', '25 N', '35 N', '30 N', '40 N', '45 N', '50 N', '55 N', '60 N', '65 N', '70 N', '75 N'],
    'HOFMs 1': [0.07, 0.21, 0.08, 0.17, 0.06, 0.09, 0.07, 0.06, 0.07, 0.07, 0.06, 0.04, 0.04, 0.02],
    'HOFMs 2': [0.25, 0.29, 0.34, 0.2, 0.2, 0.14, 0.22, 0.12, 0.14, 0.09, 0.09, 0.06, 0.12, 0.1],
    'HOFMs 3': [0.2, 0.14, 0.07, 0.11, 0.22, 0.13, 0.12, 0.11, 0.13, 0.09, 0.07, 0.1, 0.09, 0.05]
}

df2 = pd.DataFrame(data2)

# Split the data into input (X) and output (y) variables
X2 = df2[['Forces']]
y2 = df2[['HOFMs 1', 'HOFMs 2', 'HOFMs 3']]

# Convert the 'Forces' column to numeric values
X2['Forces'] = X2['Forces'].str.replace(' N', '').astype(int)

X_scaled2 = scaler.fit_transform(X2)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_scaled2, y2, epochs=200, batch_size=10, verbose=1)

# Function to predict HOFMs based on force input
def predict_hofms(force):
    force_scaled = scaler.transform([[force]])
    prediction_H = model.predict(force_scaled)[0]
    #return prediction
    mean_hofms = np.mean(prediction_H, axis=0)
    return mean_hofms


num_samples = 999  # Number of samples in the generated EMG signal
num_channels = 1  # Number of channels in the EMG signal
num_clusters = 3  # Number of clusters in the Gaussian Mixture Model

def generate_zero_mean_gmm(num_clusters, num_samples):
    # Generate covariance matrices for each cluster
    covariances = [np.eye(num_samples) for _ in range(num_clusters)]  # Identity matrix as covariance for each cluster

    # Generate weights for each cluster (uniform weights)
    weights = np.ones(num_clusters) / num_clusters

    # Generate GMM samples
    gmm_samples = np.concatenate([
        multivariate_normal.rvs(mean=np.zeros(num_samples), cov=covariances[i], size=int(num_samples * weights[i]))
        for i in range(num_clusters)
    ])

    return gmm_samples

def generate_emg_signal(force_input):
    accurate_rms = predict_rms(force_input)
    accurate_hofms = predict_hofms(force_input)
    gmm_samples = generate_zero_mean_gmm(num_clusters, num_samples)  # Generate GMM samples
    emg_signal = accurate_hofms * accurate_rms + gmm_samples  # Combine HOFMs, RMS, and GMM
    return emg_signal

# Get force input from the user
force_input = 10
l=0
while (l<4):
    emg_signal = generate_emg_signal(force_input)
    t = np.arange(num_samples)

    # Convert to compatible format
    input_array = emg_signal
    output_array = input_array * 100 + 400  # To shift range
    print(output_array.flatten())
    ++l
    time.sleep(5)  # Delay for 5 seconds before generating the next EMG signal


<ipython-input-1-2278d1b017b1>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1['Forces'] = X1['Forces'].str.replace(' N', '').astype(int)


Epoch 1/200
2/2 [==============================] - 1s 14ms/step - loss: 0.0428
Epoch 2/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0355
Epoch 3/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0290
Epoch 4/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0242
Epoch 5/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0204
Epoch 6/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0176
Epoch 7/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0154
Epoch 8/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0138
Epoch 9/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0126
Epoch 10/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0117
Epoch 11/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0109
Epoch 12/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0102
Epoch 13/200
2/2 [==============================] - 0s 7ms/s

<ipython-input-1-2278d1b017b1>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X2['Forces'] = X2['Forces'].str.replace(' N', '').astype(int)


2/2 [==============================] - 1s 11ms/step - loss: 0.0017
Epoch 2/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0014
Epoch 3/200
2/2 [==============================] - 0s 15ms/step - loss: 0.0014
Epoch 4/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0014
Epoch 5/200
2/2 [==============================] - 0s 12ms/step - loss: 0.0014
Epoch 6/200
2/2 [==============================] - 0s 10ms/step - loss: 0.0014
Epoch 7/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0014
Epoch 8/200
2/2 [==============================] - 0s 13ms/step - loss: 0.0014
Epoch 9/200
2/2 [==============================] - 0s 11ms/step - loss: 0.0014
Epoch 10/200
2/2 [==============================] - 0s 26ms/step - loss: 0.0014
Epoch 11/200
2/2 [==============================] - 0s 10ms/step - loss: 0.0014
Epoch 12/200
2/2 [==============================] - 0s 15ms/step - loss: 0.0014
Epoch 13/200
2/2 [==============================] - 0s 10ms/ste

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


...

[353.79719801 478.38280788 422.56506936 ... 406.49364457 457.82643556
 216.10342395]
1/1 [==============================] - 0s 23ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[345.08245175 486.08004307 507.2262473  ... 370.00618827 479.85449552
 384.5470986 ]
1/1 [==============================] - 0s 21ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
  plt.plot(time, emg_signal)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Generated EMG Signal')
    plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import serial
import time

# Model Definition
# Scale the input variables
scaler = MinMaxScaler()
# Build the ANN model
model = Sequential()
model.add(Dense(20, input_dim=1, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(3))  # Output layer with 3 neurons for rms 1, 2, and 3

# Load the data
data1 = {
    'Forces': ['10 N', '15 N', '20 N', '25 N', '35 N', '30 N', '40 N', '45 N', '50 N', '55 N', '60 N', '65 N', '70 N', '75 N'],
    'rms 1': [0.09, 0.22, 0.1, 0.19, 0.09, 0.11, 0.1, 0.08, 0.08, 0.09, 0.08, 0.07, 0.06, 0.05],
    'rms 2': [0.25, 0.29, 0.34, 0.2, 0.2, 0.14, 0.22, 0.12, 0.14, 0.09, 0.09, 0.06, 0.12, 0.1],
    'rms 3': [0.2, 0.14, 0.07, 0.11, 0.22, 0.13, 0.12, 0.11, 0.13, 0.09, 0.07, 0.1, 0.09, 0.05]
}

df1 = pd.DataFrame(data1)

# Split the data into input (X) and output (y) variables
X1 = df1[['Forces']]
y1 = df1[['rms 1', 'rms 2', 'rms 3']]

# Convert the 'Forces' column to numeric values
X1['Forces'] = X1['Forces'].str.replace(' N', '').astype(int)

X_scaled1 = scaler.fit_transform(X1)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_scaled1, y1, epochs=200, batch_size=10, verbose=1)

# Function to predict rms based on force input
def predict_rms(force):
    force_scaled = scaler.transform([[force]])
    prediction = model.predict(force_scaled)[0]
    mean_rms = np.mean(prediction, axis=0)
    return mean_rms

# Load the data
data2 = {
    'Forces': ['10 N', '15 N', '20 N', '25 N', '35 N', '30 N', '40 N', '45 N', '50 N', '55 N', '60 N', '65 N', '70 N', '75 N'],
    'HOFMs 1': [0.07, 0.21, 0.08, 0.17, 0.06, 0.09, 0.07, 0.06, 0.07, 0.07, 0.06, 0.04, 0.04, 0.02],
    'HOFMs 2': [0.25, 0.29, 0.34, 0.2, 0.2, 0.14, 0.22, 0.12, 0.14, 0.09, 0.09, 0.06, 0.12, 0.1],
    'HOFMs 3': [0.2, 0.14, 0.07, 0.11, 0.22, 0.13, 0.12, 0.11, 0.13, 0.09, 0.07, 0.1, 0.09, 0.05]
}

df2 = pd.DataFrame(data2)

# Split the data into input (X) and output (y) variables
X2 = df2[['Forces']]
y2 = df2[['HOFMs 1', 'HOFMs 2', 'HOFMs 3']]

# Convert the 'Forces' column to numeric values
X2['Forces'] = X2['Forces'].str.replace(' N', '').astype(int)

X_scaled2 = scaler.fit_transform(X2)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_scaled2, y2, epochs=200, batch_size=10, verbose=1)

num_samples = 999  # Number of samples in the generated EMG signal
num_channels = 1  # Number of channels in the EMG signal
num_clusters = 3  # Number of clusters in the Gaussian Mixture Model

def generate_zero_mean_gmm(num_clusters, num_samples):
    # Generate covariance matrices for each cluster
    covariances = [np.eye(num_samples) for _ in range(num_clusters)]  # Identity matrix as covariance for each cluster

    # Generate weights for each cluster (uniform weights)
    weights = np.ones(num_clusters) / num_clusters

    # Generate GMM samples
    gmm_samples = np.concatenate([
        multivariate_normal.rvs(mean=np.zeros(num_samples), cov=covariances[i], size=int(num_samples * weights[i]))
        for i in range(num_clusters)
    ])

    return gmm_samples

def generate_emg_signal(force):
    accurate_rms = predict_rms(force)
    accurate_hofms = predict_hofms(force)
    gmm_samples = generate_zero_mean_gmm(num_clusters, num_samples)  # Generate GMM samples
    emg_signal = accurate_hofms * accurate_rms + gmm_samples  # Combine HOFMs, RMS, and GMM
    return emg_signal

# Open the serial port for communication with the DAC
ser = serial.Serial('/dev/ttyUSB0', 115200)

# Get force input from the user every 5 seconds
while True:
    force_input = float(input("Enter the force value: "))
    emg_signal = generate_emg_signal(force_input)
    output_array = emg_signal * 100 + 400  # Shift and scale the values for DAC output

    for value in output_array:
        ser.write(bytes([int(value)]))
        time.sleep(0.005)

    plt.plot(emg_signal)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Generated EMG Signal')
    plt.show()


ModuleNotFoundError: ignored